### JSON (JavaScript Object Notation)
- text-based object serialization
- open standard
- human-readable
- Limited data-types
    * strings -- > "python"
    * numbers -- > 100  3.14 (all treated as floats)
    * booleans --> true, false
    * arrays (lists)
    * dcitionaries
    * empty value -- > null

In [1]:
import json

In [2]:
d1 = {'a': 100, 'b': 200}
d1_json = json.dumps(d1)
d1_json

'{"a": 100, "b": 200}'

In [3]:
d1 = {'a': 100, 'b': 200}
print(json.dumps(d1, indent=2))

{
  "a": 100,
  "b": 200
}


Notice how json.dumps turns the integer keys into string keys

In [4]:
d1 = {1: 100, 2: 200}
d1_json = json.dumps(d1)
d1_json

'{"1": 100, "2": 200}'

If we use json.loads the object that is returned is no longer equal to the object we serialized

In [5]:
d2 = json.loads(d1_json)
d1 == d2

False

In [9]:
d_json = '''
{
    "name": "John Cleese",
    "age": 82,
    "height": 1.96,
    "walksFunny": true,
    "sketches": [
        {
            "title": "Dead Parrot",
            "costars": ["Michael Palin"]
        }
    ],
    "boring": null
}
'''

d = json.loads(d_json)
d

{'name': 'John Cleese',
 'age': 82,
 'height': 1.96,
 'walksFunny': True,
 'sketches': [{'title': 'Dead Parrot', 'costars': ['Michael Palin']}],
 'boring': None}

### Serializing a tuple will turn it into a list

In [10]:
d = {'a': (1, 2, 3)}
json.dumps(d)

'{"a": [1, 2, 3]}'

### Serializing an object (sort of)

In [11]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def  __repr__(self):
        return f'Person(name={self.name}, age={self.age})'
    
    def toJSON(self):
        return dict(name=self.name, age=self.age)
    
p = Person('John', 82)
json.dumps(p.toJSON())

'{"name": "John", "age": 82}'

In [12]:
json.dumps(repr(p))

'"Person(name=John, age=82)"'

# Custom JSON Encoding

In [20]:
from datetime import datetime as dt

def format_iso(dt):
    return dt.strftime('%Y-%m-%dT%H:%M:%S')

current = dt.utcnow()
log_record = {'time': format_iso(current), 'message': 'testing'}
json.dumps(log_record)

'{"time": "2021-08-19T10:33:54", "message": "testing"}'

The above works, but is not ideal

In [22]:
def format_iso(dt):
    return dt.strftime('%Y-%m-%dT%H:%M:%S')

log_record = {'time': dt.utcnow(), 'message': 'testing'}
json.dumps(log_record, default=format_iso)

'{"time": "2021-08-19T10:36:57", "message": "testing"}'

In [38]:
def custom_json_formatter(arg):
    if isinstance(arg, dt):
        return arg.strftime('%Y-%m-%dT%H:%M:%S')
    elif isinstance(arg, set):
        return list(arg)

log_record = {
    'time': dt.utcnow(), 
    'message': 'testing',
    'args': {10, "test"},
}

json.dumps(log_record, default=custom_json_formatter)

'{"time": "2021-08-19T10:51:06", "message": "testing", "args": [10, "test"]}'

In [44]:
def custom_json_formatter(arg):
    if isinstance(arg, dt):
        return arg.strftime('%Y-%m-%dT%H:%M:%S')
    elif isinstance(arg, set):
        return list(arg)
    elif isinstance(arg, Person):
        return arg.toJSON()


class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.create_dt = dt.utcnow()
    
    def  __repr__(self):
        return f'Person(name={self.name}, age={self.age})'
    
    def toJSON(self):
        return vars(self)
        
        # Equivalent to: 
        # return {
        #     'name': self.name, 
        #     'age': self.age, 
        #     'create_dt': self.create_dt
        # }
    
    
p = Person('John', 82)

log_record = {
    'time': dt.utcnow(),
    'message': 'Created new person record',
    'person': p
}

json.dumps(log_record, default=custom_json_formatter)

'{"time": "2021-08-19T11:05:06", "message": "Created new person record", "person": {"name": "John", "age": 82, "create_dt": "2021-08-19T11:05:06"}}'

In [45]:
def custom_json_formatter(arg):
    if isinstance(arg, dt):
        return arg.strftime('%Y-%m-%dT%H:%M:%S')
    elif isinstance(arg, set):
        return list(arg)
    else:
        try:
            return arg.toJSON()
        except AttributeError:
            try:
                return vars(arg)
            except TypeError:
                return str(arg)


class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.create_dt = dt.utcnow()
    
    def  __repr__(self):
        return f'Person(name={self.name}, age={self.age})'
    
    def toJSON(self):
        return vars(self)

        
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f'Point(x={self.x}, y={self.y})'

p = Person('John', 82)
pt = Point(10, 20)

log_record = {
    'time': dt.utcnow(),
    'message': 'created new point',
    'point': pt,
    'created_by': p,
}       

json.dumps(log_record, default=custom_json_formatter)

'{"time": "2021-08-19T11:13:39", "message": "created new point", "point": {"x": 10, "y": 20}, "created_by": {"name": "John", "age": 82, "create_dt": "2021-08-19T11:13:39"}}'

In [47]:


from functools import singledispatch

@singledispatch
def json_format(arg):
    print(arg)
    try:
        return arg.toJSON()
    except AttributeError:
        try:
            return vars(arg)
        except TypeError:
            return str(arg)
        
@json_format.register(dt)
def _(arg):
    return arg.isoformat()

@json_format.register(set)
def _(arg):
    return list(arg)


class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.create_dt = dt.utcnow()
    
    def  __repr__(self):
        return f'Person(name={self.name}, age={self.age})'
    
    def toJSON(self):
        return vars(self)

        
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f'Point(x={self.x}, y={self.y})'

p = Person('John', 82)
pt = Point(10, 20)

log_record = {
    'time': dt.utcnow(),
    'message': 'created new point',
    'point': pt,
    'created_by': p,
}       

json.dumps(log_record, default=json_format)

Point(x=10, y=20)
Person(name=John, age=82)


'{"time": "2021-08-19T11:26:11.608196", "message": "created new point", "point": {"x": 10, "y": 20}, "created_by": {"name": "John", "age": 82, "create_dt": "2021-08-19T11:26:11.608196"}}'

# Custom JSON Encoding using JSONEncoder

In [48]:
class CustomJSONEncoder(json.JSONEncoder):
    def default(self, arg):
        if isinstance(arg, datetime):
            return arg.isoformat()
        else:
            super().default(arg)
            
custom_encoder = CustomJSONEncoder()
custom_encoder.encode(dt.utcnow())
        

'"2021-08-19T19:16:56.949082"'

In [49]:
class CustomJSONEncoder(json.JSONEncoder):
    def default(self, arg):
        if isinstance(arg, datetime):
            return arg.isoformat()
        else:
            super().default(arg)
            
log_record = dict(name='test', time=dt.utcnow())
json.dumps(log_record, cls=CustomJSONEncoder)

'{"name": "test", "time": "2021-08-19T19:19:38.879968"}'

In [63]:
class CustomEncoder(json.JSONEncoder):
    def __init__(self, *args, **kwargs):
        super().__init__(skipkeys=True,
                         allow_nan=False,
                         indent=2,
                         separators=(', ', ': ')
                         )
    
    def default(self, arg):
        if isinstance(arg, datetime):
            return arg.isoformat()
        else:
            return super().default(arg)
        
d = {
    'time': dt.utcnow(),
    1+1j: 'Complex',
    'name': 'Python'
}

print(json.dumps(d, cls=CustomEncoder))

{
  "time": "2021-08-19T19:49:50.486994", 
  "name": "Python"
}


In [66]:

class CustomEncoder(json.JSONEncoder):
    def __init__(self, *args, **kwargs):
        super().__init__(skipkeys=True,
                         allow_nan=False,
                         indent=2,
                         separators=(', ', ': ')
                         )
    
    def default(self, arg):
        if isinstance(arg, datetime):
            return dict(
                datatype='datetime',
                iso=arg.isoformat(),
                date=arg.date().isoformat(),
                time=arg.time().isoformat(),
                year=arg.year,
                month=arg.month,
                day=arg.day,
                hour=arg.hour,
                minutes=arg.minute,
                seconds=arg.second
            )
        else:
            return super().default(arg)
        
d = {
    'time': dt.utcnow(),
    1+1j: 'Complex',
    'name': 'Python'
}

print(json.dumps(d, cls=CustomEncoder))

{
  "time": {
    "datatype": "datetime", 
    "iso": "2021-08-19T19:50:17.809283", 
    "date": "2021-08-19", 
    "time": "19:50:17.809283", 
    "year": 2021, 
    "month": 8, 
    "day": 19, 
    "hour": 19, 
    "minutes": 50, 
    "seconds": 17
  }, 
  "name": "Python"
}


# Custom JSON Decoding

In [70]:
from fractions import Fraction

j = '''
    {
        "cake": "yummy chocolate cake",
        "myShare": {
            "objecttype": "fraction",
            "numerator": 1,
            "denominator": 8
        },
        "eaten": {
            "at": {
                "objecttype": "datetime",
                "value": "2018-10-21T21:30:00"
                },
            "time_taken": "30 seconds"
        }
    }
'''

def custom_decoder(arg):
    ret_value = arg
    if 'objecttype' in arg:
        if arg['objecttype'] == 'datetime':
            ret_value = dt.strptime(arg['value'], '%Y-%m-%dT%H:%M:%S')
        elif arg['objecttype'] == 'fraction':
            ret_value = Fraction(arg['numerator'], arg['denominator'])
    return ret_value

d = json.loads(j, object_hook=custom_decoder)
print(d)

{'cake': 'yummy chocolate cake', 'myShare': Fraction(1, 8), 'eaten': {'at': datetime.datetime(2018, 10, 21, 21, 30), 'time_taken': '30 seconds'}}


In [71]:
class Person:
    def __init__(self, name, ssn):
        self.name = name
        self.ssn = ssn
        
    def __repr__(self):
        return f'Person(name={self.name}, ssn={self.ssn})'
    
j = '''
    {
        "accountHolder": {
            "objecttype": "person",
            "name": "Eric Idle",
            "ssn": 100
        },
        "created": {
            "objecttype": "datetime",
            "value": "2018-10-21T03:00:00"
        }
    }
'''

def custom_decoder(arg):
    ret_value = arg
    if 'objecttype' in arg:
        if arg['objecttype'] == 'datetime':
            ret_value = dt.strptime(arg['value'], '%Y-%m-%dT%H:%M:%S')
        elif arg['objecttype'] == 'fraction':
            ret_value = Fraction(arg['numerator'], arg['denominator'])
        elif arg['objecttype'] == 'person':
            ret_value = Person(arg['name'], arg['ssn'])
    return ret_value

d = json.loads(j, object_hook=custom_decoder)
d

{'accountHolder': Person(name=Eric Idle, ssn=100),
 'created': datetime.datetime(2018, 10, 21, 3, 0)}

In [74]:

from decimal import Decimal

j = '''
    {
        "a": [1, 2, 3, 4, 5],
        "b": 100,
        "c": 10.5,
        "d": NaN,
        "e": null,
        "f": "python"
    }
'''

def float_handler(arg):
    print('float handler', type(arg), arg)
    return float(arg)

def int_handler(arg):
    print('int handler', type(arg), arg)
    return int(arg)

def const_handler(arg):
    print('const handler', type(arg), arg)
    return None

def obj_hook(arg):
    print('obj hook', type(arg), arg)
    return arg

def obj_pairs_hook(arg):
    print('obj pairs hook', type(arg), arg)
    return arg

json.loads(j, 
           object_hook=obj_hook,
           object_pairs_hook=obj_pairs_hook,
           parse_float=float_handler,
           parse_int=int_handler,
           parse_constant=const_handler
          )

int handler <class 'str'> 1
int handler <class 'str'> 2
int handler <class 'str'> 3
int handler <class 'str'> 4
int handler <class 'str'> 5
int handler <class 'str'> 100
float handler <class 'str'> 10.5
const handler <class 'str'> NaN
obj pairs hook <class 'list'> [('a', [1, 2, 3, 4, 5]), ('b', 100), ('c', 10.5), ('d', None), ('e', None), ('f', 'python')]


[('a', [1, 2, 3, 4, 5]),
 ('b', 100),
 ('c', 10.5),
 ('d', None),
 ('e', None),
 ('f', 'python')]

# Using JSONDecoder

In [76]:
import re

j = '''
{
    "a": 100,
    "b": 0.5,
    "rectangle": {
        "corners": {
            "b_left": {"_type": "point", "x": -1, "y": -1},
            "b_right": {"_type": "point", "x": 1, "y": -1},
            "t_left": {"_type": "point", "x": -1, "y": 1},
            "t_right": {"_type": "point", "x": 1, "y": 1}
        },
        "rotate": {"_type" : "point", "x": 0, "y": 0},
        "interior_pts": [
            {"_type": "point", "x": 0, "y": 0},
            {"_type": "point", "x": 0.5, "y": 0.5}
        ]
    }
}
'''

class CustomDecoder(json.JSONDecoder):
    base_decoder = json.JSONDecoder(parse_float=Decimal)
    
    def decode(self, arg):
        obj = self.base_decoder.decode(arg)
        pattern = r'"_type"\s*:\s*"point"'
        if re.search(pattern, arg):
            # we have at least one `Point'
            obj = self.make_pts(obj)
        return obj
    
    def make_pts(self, obj):
        # recursive function to find and replace points
        # received object could be a dictionary, a list, or a simple type
        if isinstance(obj, dict):
            # first see if this dictionary is a point itself
            if '_type' in obj and obj['_type'] == 'point':
                obj = Point(obj['x'], obj['y'])
            else:
                # root object is not a point
                # but it could contain a sub-object which itself 
                # is or contains a Point object nested at some level
                # maybe another dictionary, or a list
                for key, value in obj.items():
                    obj[key] = self.make_pts(value)
        elif isinstance(obj, list):
            # received a list - need to run each item through make_pts
            for index, item in enumerate(obj):
                obj[index] = self.make_pts(item)
        return obj
    
json.loads(j, cls=CustomDecoder)

{'a': 100,
 'b': Decimal('0.5'),
 'rectangle': {'corners': {'b_left': Point(x=-1, y=-1),
   'b_right': Point(x=1, y=-1),
   't_left': Point(x=-1, y=1),
   't_right': Point(x=1, y=1)},
  'rotate': Point(x=0, y=0),
  'interior_pts': [Point(x=0, y=0), Point(x=0.5, y=0.5)]}}